In [ ]:
import warnings
warnings.filterwarnings('ignore')

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

dataset_path = '../data/raw/emendas.json'

with open(dataset_path) as file_data:
    data = json.load(file_data)['data']
    df = pd.DataFrame(data)


# Convertendo colunas de valores para o dtype correto
value_columns = [
    'valorEmpenhado',
    'valorLiquidado',
    'valorPago',
    'valorRestoInscrito',
    'valorRestoCancelado',
    'valorRestoPago',
    'valorRestoAPagar',
]
for column in value_columns:
    df[column] = df[column].str.replace('.', '').str.replace(',', '.')    
    
value_dtypes_dict = { col:'float' for col in value_columns }

df = df.astype(value_dtypes_dict)

In [ ]:
# Sumarização do dataset
df.info()
df.sample(5)

In [ ]:
# Agrupando valorEmpenhado por localidade
loc_df = df.groupby('localidadeDoGasto')[['valorPago', 'valorEmpenhado', 'valorLiquidado']].sum()

loc_df = loc_df.drop(['Nacional', 'Nordeste'])  # Removendo outlier
loc_df = loc_df.sort_values('valorEmpenhado', ascending=False)

# Redefinindo parametros do matplotlib para melhorar legibilidade do gráfico
plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)
plt.ticklabel_format(style='plain', axis='y')

sns.barplot(
    x=loc_df.index,
    y=loc_df.valorEmpenhado,
)

# Formatting y-axis ticks with thousands separator
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

In [ ]:
# Agrupando valores por localidade
loc_df = df.groupby('localidadeDoGasto')[['valorPago', 'valorEmpenhado', 'valorLiquidado']].sum()

# Removendo outliers
loc_df = loc_df.drop(['Nacional', 'Nordeste'])

# Reorganizando dados
loc_df = loc_df.sort_values('valorEmpenhado', ascending=False)
loc_df = loc_df.reset_index()
loc_df = pd.melt(
    loc_df,
    id_vars=['localidadeDoGasto'],
    value_vars=['valorPago', 'valorEmpenhado', 'valorLiquidado'],
    var_name='tipoValor',
    value_name='valor'
)

# Redefinindo parametros do matplotlib para melhorar legibilidade do gráfico
plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)
plt.ticklabel_format(style='plain', axis='y')

sns.barplot(
    x=loc_df.localidadeDoGasto,
    y=loc_df.valor,
    hue=loc_df.tipoValor,
)

# Formatting y-axis ticks with thousands separator
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

In [ ]:
# Agrupando valores por ano
year_df = df.groupby('ano')[['valorPago', 'valorEmpenhado', 'valorLiquidado']].sum()

# Reorganizando dados
year_df = year_df.reset_index()
year_df = pd.melt(
    year_df,
    id_vars=['ano'],
    value_vars=['valorPago', 'valorEmpenhado', 'valorLiquidado'],
    var_name='tipoValor',
    value_name='valor'
)

plt.figure(figsize=(10, 4))

sns.lineplot(
    x=year_df.ano,
    y=year_df.valor,
    hue=year_df.tipoValor,
    style=year_df.tipoValor,
    markers=True
)

# Formatting y-axis ticks with thousands separator
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])